# <font color="green"> Inception-v3 <font>

This code are based on the article https://distill.pub/2017/feature-visualization/

In [3]:
import tensorflow as tf
import numpy as np
import os
import os.path
import transfer_learning_v3 # This module is from homework4 and we just made several changes to the return values of _create_model() 
import pandas as pd
from scipy.misc import imsave
from scipy.stats import rankdata
import time
import fnmatch
from PIL import Image
from math import ceil, sqrt

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
import keras

Using TensorFlow backend.


## 1. Parser tensor from pretrained model

In [ ]:
def parser_tensor(layer_name,model_dir):
    """input: the output of layer you want"""
    with tf.Graph().as_default() as graph:
        model_path=os.path.join(model_dir,"classify_image_graph_def.pb")
        with tf.gfile.FastGFile

In [ ]:
def _create_model_graph() -> typing.Tuple[tf.Graph, tf.Tensor, tf.Tensor, tf.Tensor]:
    """"Creates a graph from saved GraphDef file and returns a Graph object.

    This creates the graph for the Inception model that was downloaded.

    Returns:
      Graph holding the trained Inception network, and various tensors we'll be
      manipulating.
    """
    with tf.Graph().as_default() as graph:
        model_path = os.path.join(_MODEL_DIR, 'classify_image_graph_def.pb')
        with tf.gfile.FastGFile(model_path, 'rb') as f:
            graph_def = tf.GraphDef()
            graph_def.ParseFromString(f.read())
            bottleneck, resized_input, softmax,conv,conv1,conv2,pool,conv3,conv4,pool1,conv_W,conv1_W,conv2_W,conv3_W,conv4_W= (tf.import_graph_def(
                graph_def,
                name='',
                return_elements=[
                    'pool_3/_reshape:0',  # bottleneck tensor name
                    'Mul:0',  # resized input tensor name
                    'softmax:0',  # predicted probability tensor name
                    "conv:0",
                    "conv_1:0",
                    "conv_2:0",
                    'pool:0',
                    "conv_3:0",
                    "conv_4:0",
                    'pool_1:0',
                    "conv/conv2d_params:0",
                    "conv_1/conv2d_params:0",
                    "conv_2/conv2d_params:0",
                    "conv_3/conv2d_params:0",
                    "conv_4/conv2d_params:0",
                ]))
    return graph, bottleneck, resized_input, softmax,conv,conv1,conv2,pool,conv3,conv4,pool1,conv_W,conv1_W,conv2_W,conv3_W,conv4_W

### 1. Neuron

### 2. Channel: Feature Maps

In [ ]:
def _deepdream(graph, sess, op_tensor, X, feed_dict, layer, path_outdir, path_logdir):
    tensor_shape = op_tensor.get_shape().as_list()

    with graph.as_default() as g:
        n = (config["N"] + 1) // 2
        feature_map = tf.placeholder(dtype = tf.int32)
        tmp1 = tf.reduce_mean(tf.multiply(tf.gather(tf.transpose(op_tensor),feature_map),tf.diag(tf.ones_like(feature_map, dtype = tf.float32))), axis = 0)
        tmp2 = 1e-3 * tf.reduce_mean(tf.square(X), axis = (1, 2 ,3))
        tmp = tmp1 - tmp2
        t_grad = tf.gradients(ys = tmp, xs = X)[0]

        with sess.as_default() as sess:
            input_shape = sess.run(tf.shape(X), feed_dict = feed_dict)
            tile_size = input_shape[1 : 3]
            channels = input_shape[3]

            lap_in = tf.placeholder(np.float32, name='lap_in')
            laplacian_pyramid = lap_normalize(lap_in, channels, scale_n=config["NUM_LAPLACIAN_LEVEL"])

            image_to_resize = tf.placeholder(np.float32, name='image_to_resize')
            size_to_resize = tf.placeholder(np.int32, name='size_to_resize')
            resize_image = tf.image.resize_bilinear(image_to_resize, size_to_resize)

            end = len(units)
            for k in range(0, end, n):
                c = n
                if k + n > end:
                    c = end - ((end // n) * n)
                img = np.random.uniform(size = (c, tile_size[0], tile_size[1], channels)) + 117.0
                feed_dict[feature_map] = units[k : k + c]

                for octave in range(config["NUM_OCTAVE"]):
                    if octave > 0:
                        hw = np.float32(img.shape[1:3])*config["OCTAVE_SCALE"]
                        img = sess.run(resize_image, {image_to_resize : img, size_to_resize : np.int32(hw)})

                        for i, im in enumerate(img):
                            min_img = im.min()
                            max_img = im.max()
                            temp = denoise_tv_bregman((im - min_img) / (max_img - min_img), weight = config["TV_DENOISE_WEIGHT"])
                            img[i] = (temp * (max_img - min_img) + min_img).reshape(img[i].shape)

                    for j in range(config["NUM_ITERATION"]):
                        sz = tile_size
                        h, w = img.shape[1:3]
                        sx = np.random.randint(sz[1], size=1)
                        sy = np.random.randint(sz[0], size=1)
                        img_shift = np.roll(np.roll(img, sx, 2), sy, 1)
                        grad = np.zeros_like(img)
                        for y in range(0, max(h-sz[0]//2,sz[0]), sz[0] // 2):
                            for x in range(0, max(h-sz[1]//2,sz[1]), sz[1] // 2):
                                    feed_dict[X] = img_shift[:, y:y+sz[0],x:x+sz[1]]
                                    try:
                                        grad[:, y:y+sz[0],x:x+sz[1]] = sess.run(t_grad, feed_dict=feed_dict)
                                    except:
                                        pass

                        lap_out = sess.run(laplacian_pyramid, feed_dict={lap_in:np.roll(np.roll(grad, -sx, 2), -sy, 1)})
                        img = img + lap_out
                is_success = write_results(img, (layer, units, k), path_outdir, path_logdir, method = "deepdream")
                print("%s -> featuremap completed." % (", ".join(str(num) for num in units[k:k+c])))
    return is_success


### 3. Layer

### 4. Class Logits

### 5. Class probability